**Definition:** RDD Transformations are operations on RDDs that result into new RDDs.  
No updates happen to the existing RDDs (they are by default immutable).  
Spark retains RDD lineage, using an operator graph. 

RDD Transformations are lazy operations.  
We can distinguish them to:
- Narrow transformations: they compute results that live in the same partition as the original RDD. They impose no data movement between partitions. Examples are 'map' and 'filter'.
- Wider transformations: they compute results that live in many partitions. Data movement between partitions is possible. Examples are 'groupByKey' and 'reduceByKey'.

In [1]:
from pyspark import SparkContext
import os
os.chdir('/Users/chkapsalis/Documents/GitHub/Big_Data_Architectures/3_Spark')

# For some reason i need to run this every time in order to get it work
import os
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk@11/libexec/openjdk.jdk/Contents/Home" 

sc = SparkContext("local[1]", "app")


25/03/25 15:09:04 WARN Utils: Your hostname, ChristoorossAir resolves to a loopback address: 127.0.0.1; using 192.168.1.18 instead (on interface en0)
25/03/25 15:09:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/25 15:09:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/25 15:09:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/03/25 15:09:05 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
# Basic Transformations
nums = sc.parallelize([1,2,3])

# Pass each element through a function
squares = nums.map(lambda x: x * x)  # => [1, 4, 9]

# Keep elements passing a predicate
even = squares.filter(lambda x: x % 2 == 0)  # => [4]

# Map each element to zero or more others
nums.flatMap(lambda x: range(0, x))   # => [0,0,1,0,1,2]

PythonRDD[1] at RDD at PythonRDD.scala:53

In [3]:
# RDD Filtering - ".filter()" filters an RDD based on a given predicate
rdd = sc.parallelize(range(20))
rdd1 = rdd.filter(lambda x: x % 2 == 0)
print(rdd1.collect())

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


In [4]:
# RDD Map - ".map()" maps a function to each element of an RDD
rdd2 = rdd.map(lambda x: (x, x % 2 == 0))
print(rdd2.collect())

[(0, True), (1, False), (2, True), (3, False), (4, True), (5, False), (6, True), (7, False), (8, True), (9, False), (10, True), (11, False), (12, True), (13, False), (14, True), (15, False), (16, True), (17, False), (18, True), (19, False)]


In [5]:
nums = sc.parallelize([1,2,3,4,5])
sq = nums.map(lambda x: x * x).collect()
for n in sq:
    print(n)

1
4
9
16
25


In [7]:
# RDD flatMap - ".flatMap()" maps a function to each element of the RDD and flattens the RDD
rdd = sc.parallelize([[1,2,3], [4,5,6], [7,8,9]])  # it will take in each sub-list as an rdd, and will create an rdd made of these 3 sub-rdds
rdd1 = rdd.map(lambda x: x[-1::-1])  # so it will revert the current ordering of elements into each sub-list / sub-rdd
print(rdd1.collect())
rdd2 = rdd.flatMap(lambda x: x[-1::-1])
print(rdd2.collect())

[[3, 2, 1], [6, 5, 4], [9, 8, 7]]
[3, 2, 1, 6, 5, 4, 9, 8, 7]


More Transformations:
- **map(func)**: returns a new distributed dataset (aka an rdd) by passing each element of the source through a function func
- **filter(func)**: returns a new dff formed by selecting those elements of the source on which func returns true
- **flatMap(func)**: similar to map, but each input item can be mapped to 0 or more output items
- **union(otherDataset)**: return a new rdd that contains the union of the elements in the source dataset and the argument
- **intersection(otherDataset)**: returns a new rdd that contains the intersection of elements in the source dataset and the argument
- **groupByKey([numPartitions])**: when called on a dataset of (k,v) pairs, returns a dataset of (K, iterable<V>) pairs
- **reduceByKey(func, [numPartitions])**: when called on a dataset of (K,V) pairs, returns a dataset of (K,V) pairs where the values for each key are aggregated using the given reduce function func, which must be of type (V, V) => V. Like in groupByKey, the number of reduce tasks is configurable through an optional second argument.
- **join(otherDataset, [numPartitions])**: when called on datasets of type (K,V) and (K,W), returns a dataset of (K, (V, W)) pairs with all pairs of elements for each key. Outer joins are supported through leftOuterJoin, rightOuterJoin, and fullOuterJoin

## RDD Actions
- Definition: they are operations that return the raw values
- Any RDD method that returns anything other than an RDD is an RDD Action

In [ ]:
# Basic Actions
nums = sc.parallelize([1,2,3])

# Retrieve RDD contents as a local collection
nums.collect()  # => [1,2,3]

# Returns an ARRAY of the first first K elements
nums.take(2)  # => [1,2]

# Count number of elements
nums.count()  # => 3

# Merge elements with an associative function
nums.reduce(lambda x, y: x+y)  # => 6

# Write elements to a text file
nums.saveAsTextFile('file:///' + os.getcwd() + '/file.txt')  # or 'hdfs:///file.txt' if i did not want a local copy but one on hdfs

6

Apache Spark's fold(), reduce(), and aggregate() operations provide flexible ways   
to process distributed data in RDDs. Let’s analyze their behavior and use cases.  

## RDD Fold Operation
The fold() action aggregates elements within partitions first, then combines partition results.
### Key characteristics:
- Requires a neutral element (zeroValue) that serves as:
- Initial value for partition-level aggregation
- Initial value for combining partition results
- **Guarantees a result even for empty RDDs**
- **Must use associative operations**

Example breakdown:

In [14]:
rdd = sc.parallelize([1,3,6,8,3,4,6,7], 3)  # 3 partitions minimum (as a hint; NOT guaranteed!)
sum = rdd.fold(0, lambda x,y: x+y)  # Neutral element = 0 (identity for addition)
print(sum)

38


Execution logic:
- Partition 1: 0 + 1 = 1
- Partition 2: 0 + 3 + 6 = 9
- Partition 3: 0 + 8 + 3 + 4 + 6 + 7 = 28
- Combine results: 0 + (1) + (9) + (28) = 38

In [15]:
# RDD Fold - another example
rdd = sc.parallelize([1,3,6,8,3,4,6,7])  # no # partitions specified - it will be automatically assigned the number of cores on our local machine 
sum = rdd.fold(0, lambda x, y: x + y)  # zero is the neutral element of summation - it does NOT affect the sum 

# For the following two fold operations, I am going to provide (at random) the first element of each partition as the initial value 
# I do not give it float('inf') or None, because spark applies the zeroValue to every partition, so the zeroValue must be of the SAME TYPE as the DATA
# So if we did zeroValue=None, it would raise a TypeError because None and int cannot be compared with min()
minv = rdd.fold(rdd.take(1)[0], lambda x, y: min(x, y))  # .take(1) returns an array with a single element, so we do .take(1)[0] to retrieve the element itself
maxv = rdd.fold(rdd.take(1)[0], lambda x, y: max(x, y))

print(f"Min: {minv}, Max: {maxv}, Sum: {sum}")

Min: 1, Max: 8, Sum: 38


In [16]:
# We could perform the exact same action using the '.reduce()' method

rdd = sc.parallelize([1,3,6,8,3,4,6,7])
sum = rdd.reduce(lambda x, y: x + y)  # No initial value !!! 
minv = rdd.reduce(lambda x, y: min(x, y))
maxv = rdd.reduce(lambda x, y: max(x, y))

print(f"Min: {minv}, Max: {maxv}, Sum: {sum}")

Min: 1, Max: 8, Sum: 38


## 📌 .reduce()  
- **Pros:** More efficient when neutral element isn't needed
- **Cons:** Fails on empty RDDs
- **Requires** associative and commutative operations
#### If the RDD we try to digest is empty, .reduce() will throw an error.


## 📌 .fold()  
Use when:
- Handling empty RDDs (returns zeroValue) 
- Custom initial states are needed 
- Can also work with non-commutative operations 

**If the RDD we try to digest is empty, .fold() will not throw an error; it will return the specified initial value**


In [17]:
# Key-value operations

pets = sc.parallelize([('cat', 1), ('dog', 1), ('cat', 2)])

pets.reduceByKey(lambda x, y: x+y)
# => [('cat',3), ('dog', 1)]

pets.groupByKey()
# => [('cat', [1,2]), ('dog', [1])]

pets.sortByKey()  # by default, the underlying iterators (when the single elements of the RDD are iterators) are compared based on their first element
# and if that is the same, then by the second element, etc 
# => [('cat', 1), ('cat', 2), ('dog', 1)]
# Moreover, rdd.sortByKey() by default sets 'ascending=True'!

PythonRDD[41] at RDD at PythonRDD.scala:53

### RDD Aggregate - ".aggregate()" operates on RDD partitions 
It **removes .fold()'s type constraint** by allowing:  
- Different input/output types  
- Separate functions for partition aggregation (seqOp) and result merging (combOp)  

The aggregate function first aggregates elements in each partition and then aggregates results of all partitions to get the final result (like the reducebykey or fold).  
It takes 3 parameters:
- An initial value for the aggregation of each partition
- A function that takes two parameters of the type of the RDD elements AND returns a result of POSSIBLY DIFFERENT TYPE
- A function that takes two parameters of the type of the result of the RDD aggregations and returns the final result 

In [18]:
rdd = sc.parallelize([1,2,4,5,2,4,6,1])  # when we do not define the number of partitions, it automatically becomes equal to the number of cpu cores on our machine
# here we will compute the sum of element values across all partitions
r = rdd.aggregate(
    0,   # Neutral element of addition
    (lambda acc, v: acc + v),   # seqOp (partition aggregation) -> arguments: accumulator, value
    (lambda acc1, acc2: acc1+acc2)    # combOp (result merging)  -> arguments: accumulator1, accumulator2 - the fact that we use 2 accumulators has nothing to do 
    # with using 2 partitions - it is just the standard way of expressing this associative operation
)

print(f"Output {r}")

Output 25


In [20]:
# Real-world use case: Calculating average with sum and count:   [the FINAL result of .aggregate() will comprise of 2 values => returned in a tuple]
(sum_val, count) = rdd.aggregate(
    (0,0),  # one of the initialization values will represent the running sum of the values across partitions, and the other will represent a running count of the values we come across
    lambda acc,v: (acc[0]+v, acc[1]+1),  # Update sum and count per element
    lambda acc1,acc2: (acc1[0]+acc2[0], acc1[1]+acc2[1])  # Combine results
)

avg = sum_val / count

In [21]:
print(avg)

3.125


In [22]:
# Another aggregate example
rdd = sc.parallelize([("A",10),("B",20),("B",30),("C",40),("D",30),("E",60)])
r = rdd.aggregate(
    0,
    lambda acc, v: acc + v[1],
    lambda acc1, acc2: acc1 + acc2
)
print(f"Output {r}")

Output 190


In [23]:
# RDD ForEach

rdd = sc.parallelize(range(5))
rdd.foreach(lambda x: print('Element {0}'.format(x)))

Element 0
Element 1
Element 2
Element 3
Element 4


In [ ]:
# RDD saveAsTextFile
# ATTENTION: it expects it in a full path preceded by "file:" to accept it
rdd.saveAsTextFile('file:/Users/chkapsalis/Documents/GitHub/Big_Data_Architectures/3_Spark/out.txt')

### Further RDD Actions
- **reduce(func)**: Aggregate the elements of the dataset using a function func (which takes two arguments and returns one). The function should be commutative and associative so that it can be computed correctly in parallel.
- **collect()**: return all the elements of the dataset as an array at the driver program. This is typically useful after a filter or other operation that returns a sufficiently small subset of the data.
- **count():** return the number of elements in the dataset
- **first():** return the first element of the dataset (similar to take(1)[0[)
- **take(n):** return **AN ARRAY** with the first n elements of the dataset
- **saveAsTextFile(path)**: write the elements of the dataset as a text file (or set of text files) in a given directory in the local filesystem, HDFS or any other Hadoop-supported file system. Spark will call toString on each element to convert it to a line of text in the file.
- **countByKey()**: returns a dictionary with keys the values of the dataset and values the number of their occurrences
- **foreach(func)**: run a function func on each element of the dataset. This is usually done for side effects such as updating an Accumulator or interacting with external storage systems. 

In [27]:
# Advanced Example: Word Count
lines = sc.textFile('file:///' + os.getcwd() + '/hamlet.txt')  # !!! this is the way we need to input files in order for the sparkContext.textFile() method to work
counts = lines.flatMap(lambda line: line.split(' ')) \
            .map(lambda word: (word, 1)) \
            .reduceByKey(lambda x, y: x + y) \
            .sortBy(lambda kv: kv[1], ascending=False) \
            .take(10)#.collect()

# so the previous takes in the input file, 
# breaks each line into a (flat) list of words,
# calculates the total count of occurrences for each word,
# sorts (word, total_count) pairs by the total_count value in ascending order,
# shows the words with the top 10 total_count values

In [28]:
print(counts)

[('', 20380), ('the', 929), ('and', 680), ('of', 625), ('to', 608), ('I', 523), ('a', 453), ('my', 444), ('in', 382), ('you', 361)]


```


```

# Accumulator and Broadcast Variables in Spark

## Theory
Functions that are passed to methods like map() or filter() can use variables defined in the global scope (i.e., the driver program).  
Each task running on the cluster gets a new copy of each variable, and updates from these copies are not propagated back to the driver.  
Spark automatically sends all variables referenced in closures to the worker nodes.  
However, this can be inefficient because the default task launching mechanism is optimized for small task sizes. Moreover, the same variable may be used in multiple parallel operations, but Spark will send it separately for each operation.

So there exist two common types of communication patterns that help prevent bottlenecks in the efficiency of a Spark application:
1. aggregation of results
2. broadcasts

**Accumulators** provide a mechanism for aggregating values from worker nodes back to the driver program.   
**Broadcast** variables allow the application to **efficiently** send large, read-only values to all worker nodes for use in RDD operations.  

### Broadcast Variables
**Definition:** Broadcast variables are read-only shared variables that are cached and available on all nodes of a cluster.   
Instead of sending this data with every task, Spark uses efficient broadcast algorithms to distribute broadcast variables.  
They bring significant efficiency **enhancements when large, read-only lookup tables or even a large feature vector in an ML algorithm must be sent to all nodes.** (when to use)  
Spark automatically sends all variables referenced in closures to the worker nodes.


In [29]:
# the following 'colors' list is a typical case of a lookup table
colors = {
    'blue': (0,0,255),
    'green': (0,255,0),
    'red': (255,0,0),
    'yellow': (255,255,0),
    'white': (255,255,255),
    'black': (0,0,0)    
}

# I do not reference this 'bcolor' broadcast variable in my application; it just serves the whole operation of caching 'colors' so it can be efficiently broadcast 
# to all the nodes referencing it. 
bcolor = sc.broadcast(colors)

In [30]:
articles = [
    ('table', 'green'),
    ('fridge', 'white'),
    ('TV', 'black'),
    ('book', 'yellow')
]

rdd = sc.parallelize(articles)

In [34]:
rdd1 = rdd.map(lambda x: (x[0], colors[x[1]]))
print(rdd1.collect())

[('table', (0, 255, 0)), ('fridge', (255, 255, 255)), ('TV', (0, 0, 0)), ('book', (255, 255, 0))]


## Accumulator Variables

**Definition:** Accumulator variables are shared variables used to accumulate a **value of any numeric type over associative and commutative operations**. Custom accumulator types may also be defined.  
One of **the most common uses** of accumulators is to count events that occur during job execution for debugging purposes.  
**Tasks cannot read the accumulator** value; **only the driver program** can read it, **using its '.value()' method**.

In [35]:
# First example - numeric summation
acc = sc.accumulator(0)  # initial value is set to zero in this use case

rdd = sc.parallelize([1,3,5,7,9])
rdd.foreach(lambda x: acc.add(x))

print(f'Sum of odds: {acc}')

Sum of odds: 25


In [36]:
# Second example: string concatenation - this makes for a case where we define our own, custom accumulator variable (since by default accs only support numeric values)
# For this reason, we will need to extend on the behavior (aka methods) of the default AccumulatorParam class !!! 
from pyspark.accumulators import AccumulatorParam

class strAcc(AccumulatorParam):
    def zero(self, _):  # initial value of the accumulator variable ! 
        return ''
    def addInPlace(self, var, val):  # this defines how the `.add()` method will behave
        return var + val

accs = sc.accumulator('', strAcc())  
rdd1 = sc.parallelize(['a','b','c','d','e'])
rdd1.foreach((lambda x: accs.add(x)))

print(f'Letter concats: {accs}')

Letter concats: abcde


## Partitioning and Repartitioning

### Partitioning 
- **Local partitioning:** when running on local in standalone mode, Spark partitions data into the number of CPU cores or the value specified when SparkContext is created
- **Cluster partitioning:** when running on HDFS cluster mode, spark creates one partition for each block of the file. The number of partitions will be equal to max(total number of cores on all executor nodes in a cluster, 2).

### Repartitioning
Repartition is **used to increase or decrease the RDD partitions**.

### Coalesce 
**Coalesce** is used to only decrease the number of partitions **in an efficient way**.

### RDD Shuffling
**Definition:** RDD Shuffling is a mechanism for redistributing or repartitioning data so that the data get grouped differently across partitions, across different executors, and even across machines. 
It is an expensive operation as it moves the data between executors or even between worker nodes in a cluster. This leads to heavy Disk I/O, it involves the costly operations of data serialization and deserialization, as well as Network I/O.  
**Some of the operations that trigger shuffling** are 'repartition()', 'coalesce()', 'groupByKey()', 'reduceByKey()', 'cogroup()', 'join()'.  
Shuffling is **NOT triggered by 'countByKey()'**.


# Working with Multiple Datasets in Spark

In [2]:
visits = sc.parallelize([
    ("index.html", "1.2.3.4"),
    ("about.html", "3.4.5.6"),
    ("index.html", "1.3.3.1")
])

pageNames = sc.parallelize([
    ("index.html", "Home"),
    ("about.html", "About")
])

### RDD Join & Cogroup in PySpark
Both .join() and .cogroup() are used for combining data from two RDDs, but they serve different purposes and behave differently in their output structure.  
1. **join()**
    - **Behavior:** Performs an inner join on two RDDs based on their keys.
    - **Returns:** A new RDD where each key is associated with a tuple of values from both RDDs.
    - **Structure:** (key, (value_from_rdd1, value_from_rdd2))
    - **Ideal for:** One-to-one or one-to-many relationships where you need pairwise matching of elements, or relational-style lookups (e.g., mapping user visits to webpage names).


2. **cogroup()**
    - **Behavior:** Groups values from both RDDs by key but does not directly pair them like join().
    - **Returns:** A new RDD where each key is associated with two result iterables:
        - The first iterable contains values from RDD1.
        - The second iterable contains values from RDD2.
    - **Structure:** (key, (iterable_from_rdd1, iterable_from_rdd2))
    - **Ideal for:** Many-to-many relationships or custom aggregation; cases where you need to group values before processing, rather than directly joining them; cases where you want to aggregate values before performing an operation

In [4]:
### RDD Join implementation

visits.join(pageNames).collect()

[('about.html', ('3.4.5.6', 'About')),
 ('index.html', ('1.2.3.4', 'Home')),
 ('index.html', ('1.3.3.1', 'Home'))]

In [5]:
### RDD Cogroup implementation

visits.cogroup(pageNames).collect()

[('about.html',
  (<pyspark.resultiterable.ResultIterable at 0x1229e6e00>,
   <pyspark.resultiterable.ResultIterable at 0x1229e6b60>)),
 ('index.html',
  (<pyspark.resultiterable.ResultIterable at 0x1229e54e0>,
   <pyspark.resultiterable.ResultIterable at 0x1229e6c80>))]

In [6]:
# To better view the iterables matched to each key:
result = visits.cogroup(pageNames).mapValues(lambda x: (list(x[0]), list(x[1]))).collect()
print(result)

[('about.html', (['3.4.5.6'], ['About'])), ('index.html', (['1.2.3.4', '1.3.3.1'], ['Home']))]
